In [37]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D
from keras.layers.pooling import MaxPooling2D
import keras
import os
import numpy as np
from sklearn import model_selection
from PIL import Image
from keras.utils import np_utils

def travel(orgpth,img=[],label=[]):
    fs = os.listdir(orgpth)  
    for f in fs:
        tmp_path = os.path.join(orgpth, f)
        if not os.path.isdir(tmp_path):  
#             print('do with file: %s'%tmp_path)
            img_0,label_0 = processfile(tmp_path)
            img.append(img_0)
            label.append(label_0)
        else:
#             print('do with dir: %s'%tmp_path)
            travel(tmp_path,img,label)
    
def readarray(im_pth,img=[],label=[]):   
    travel(im_pth,img,label)
    n_img = np.array(img)
    return n_img,label

def labelprocess(y):
    for index, value in enumerate(y):
        if value=='axes':y[index]=0
        if value=='boots':y[index]=1
        if value=='carabiners':y[index]=2          
        if value=='crampons':y[index]=3
        if value=='gloves':y[index]=4
        if value=='hardshell_jackets':y[index]=5
        if value=='harnesses':y[index]=6
        if value=='helmets':y[index]=7
        if value=='insulated_jackets':y[index]=8
        if value=='pulleys':y[index]=9
        if value=='rope':y[index]=10
        if value=='tents':y[index]=11
    y = np_utils.to_categorical(y, 12) 

def processfile(im_pth):
    im = Image.open(im_pth)
    imarray = np.asarray(im, dtype=np.uint8)
    n_imarray = imarray.flatten()
    label = im_pth.split('/')[-2]
#     pca = PCA(n_components=2)  
#     n_imarray = pca.fit_transform(imarray)
#     PCA(copy=True, n_components=2, whiten=False) 
#     print(n_imarray,n_imarray.shape)
#     print(im_pth.split('/')[-2])
#     print(flatten_im.shape,n_imarray.shape)
    return n_imarray,label

def Kerasmodel(x,y,xx,yy):
    model = Sequential() 
  
    model.add(Dense(512, input_shape=(128,128,3)))  
    model.add(Conv2D(32, (4, 4), input_shape=(128,128,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(5, 5)))
    model.add(Conv2D(32, (4, 4)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(5, 5))) 
    model.add(Dense(1, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

    # x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
    model.fit(x, y, epochs=5, batch_size=32)
#     model.fit(x, y, epochs=5, batch_size=32, verbose=1, validation_data=(xx, yy))

    # model.train_on_batch(x_batch, y_batch)
    loss_and_metrics = model.evaluate(xx, yy, batch_size=32)
    classes = model.predict(xx, batch_size=32)
    
    return loss_and_metrics, classes


im_pth = "gear_images_a"
readarray(im_pth)
img,label = readarray(im_pth)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(img, label, test_size=0.1, random_state=0)
labelprocess(Y_train)
labelprocess(Y_test)
loss_and_metrics, classes = Kerasmodel(X_train, Y_train, X_test, Y_test)
print(loss_and_metrics, '\n', classes)



ValueError: Error when checking input: expected dense_57_input to have 4 dimensions, but got array with shape (3819, 97200)